In [496]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [497]:
df = pd.read_csv("Fallvilt_beriket_med_vær.csv",sep=";", low_memory=False)

In [498]:
df=df[df['Art'].isin(['Elg', 'Hjort', 'Rådyr'])].copy()  
#df=df[df['Art'].isin(['Hjort'])].copy()  
df=df[df['vegkategori'].isin(['E','F','K'])].copy()
df=df[df['År']==2025].copy()

In [499]:
df.columns

Index(['Dato', 'År', 'Kommune', 'Stedfesting', 'Art', 'Kjønn', 'Alder',
       'Årsak', 'Utfall', 'Merkelappnummer', 'Fallvilt-ID', 'UTM33 øst',
       'UTM33 nord', 'vegsystemreferanse.kortform', 'vegkategori', 'fase',
       'vegnr', 'strekning', 'delstrekning', 'arm', 'adskilte_løp',
       'trafikantgruppe', 'retning', 'meter', 'veglenkesekvensid',
       'relativPosisjon', 'veglenkesekvens.kortform', 'geometri.wkt',
       'geometri.srid', 'kommune (treff)', 'avstand_vegnettet_m',
       'Vegobjekt_540_id', 'ÅDT, total', 'Vegobjekt_105_id', 'Fartsgrense',
       'Vegobjekt_540_lengde', 'snow_depth', 'max_temperature',
       'min_temperature', 'mean_temperature', 'total_precipitation',
       'max_wind_speed', 'mean_wind_speed', 'max_wind_gust',
       'precipitation_type'],
      dtype='object')

In [500]:
df=df[['Vegobjekt_105_id','Dato','Art','ÅDT, total','Vegobjekt_540_lengde','adskilte_løp', 'snow_depth']].dropna().copy()

In [501]:
df["Dato"] = pd.to_datetime(df["Dato"]).copy()

In [502]:
def maaned_til_arstid(m):
    if m in [12, 1, 2]:
        return "Vinter"
    elif m in [3, 4, 5]:
        return "Vår"
    elif m in [6, 7, 8]:
        return "Sommar"
    else:
        return "Haust"

In [503]:
df["årstid"] = df["Dato"].dt.month.apply(maaned_til_arstid)
df["årstid"] = df["årstid"].astype("category").copy()

In [536]:
df

KeyError: 'HendelsesDatoTid'

In [505]:
df["snow_depth_cm"] = (
    df["snow_depth"]
    .str.replace(" cm", "", regex=False)
    .astype(float).copy()
)

In [506]:
df=df[df['snow_depth_cm']>=0].copy()

In [507]:
df["snø"] = (df["snow_depth_cm"] > 0).astype(int)
df["snø"] = df["snø"].map({0: "Ikkje snø", 1: "Snø"})
df["snø"] = df["snø"].astype("category")
df["Art"] = df["Art"].astype("category")
#df["adskilte_løp"] = df["adskilte_løp"].astype("category")

In [508]:
df=df[['Vegobjekt_105_id','ÅDT, total','Vegobjekt_540_lengde','Art','snø','årstid']].copy()

In [509]:
df

,Vegobjekt_105_id,"ÅDT, total",Vegobjekt_540_lengde,Art,snø,årstid
236,8.697445e+07,700.0,17818.619,Rådyr,Snø,Vinter
241,8.697566e+07,697.0,28148.139,Rådyr,Snø,Vinter
245,8.531025e+07,300.0,3913.985,Rådyr,Snø,Vinter
248,8.531139e+07,2230.0,10162.666,Hjort,Snø,Vinter
249,8.531191e+07,280.0,3714.520,Rådyr,Ikkje snø,Vinter
...,...,...,...,...,...,...
2931,1.016391e+09,2450.0,21968.428,Hjort,Snø,Vinter
2941,7.281254e+08,1600.0,11320.552,Elg,Snø,Vinter
2944,7.281254e+08,1600.0,11320.552,Elg,Snø,Vinter
2954,7.281254e+08,1600.0,11320.552,Elg,Snø,Vinter


In [510]:
df["eksponering"] = (
    df["ÅDT, total"]
    * 365
    * df["Vegobjekt_540_lengde"]
    / 100_000
)

df["log_eksponering"] = np.log(df["eksponering"])


In [511]:
df

,Vegobjekt_105_id,"ÅDT, total",Vegobjekt_540_lengde,Art,snø,årstid,eksponering,log_eksponering
236,8.697445e+07,700.0,17818.619,Rådyr,Snø,Vinter,45526.571545,10.726051
241,8.697566e+07,697.0,28148.139,Rådyr,Snø,Vinter,71610.273023,11.178994
245,8.531025e+07,300.0,3913.985,Rådyr,Snø,Vinter,4285.813575,8.363066
248,8.531139e+07,2230.0,10162.666,Hjort,Snø,Vinter,82719.019907,11.323205
249,8.531191e+07,280.0,3714.520,Rådyr,Ikkje snø,Vinter,3796.239440,8.241766
...,...,...,...,...,...,...,...,...
2931,1.016391e+09,2450.0,21968.428,Hjort,Snø,Vinter,196452.667390,12.188177
2941,7.281254e+08,1600.0,11320.552,Elg,Snø,Vinter,66112.023680,11.099106
2944,7.281254e+08,1600.0,11320.552,Elg,Snø,Vinter,66112.023680,11.099106
2954,7.281254e+08,1600.0,11320.552,Elg,Snø,Vinter,66112.023680,11.099106


In [512]:
df_agg = (
    df
    .groupby(
        ["Vegobjekt_105_id","årstid"],
        observed=True,      # fjern FutureWarning
        as_index=False
    )
    .agg(
        antall_kollisjoner=("Vegobjekt_105_id", "count"),
        log_eksponering=("log_eksponering", "first")
    )
)


In [513]:
df_agg

,Vegobjekt_105_id,årstid,antall_kollisjoner,log_eksponering
0,7.870234e+07,Haust,1,2.173430
1,7.870594e+07,Vår,1,2.169558
2,7.870722e+07,Vinter,1,5.884972
3,7.870970e+07,Vår,1,7.007342
4,7.956042e+07,Vinter,1,5.968744
...,...,...,...,...
470,1.019155e+09,Haust,2,8.971983
471,1.019155e+09,Sommar,1,8.971983
472,1.022004e+09,Haust,1,10.372474
473,1.022466e+09,Vinter,1,10.822477


In [514]:
# 1. Summen skal stemme
df_agg["antall_kollisjoner"].sum() == len(df)

True

In [515]:
# 2. Ingen NaN
df_agg.isna().sum()

Vegobjekt_105_id      0
årstid                0
antall_kollisjoner    0
log_eksponering       0
dtype: int64

In [516]:
# 3. Fornuftige gruppestorleikar
df_agg["antall_kollisjoner"].describe()

count    475.000000
mean       1.458947
std        1.139944
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max       10.000000
Name: antall_kollisjoner, dtype: float64

In [517]:
#f_agg=df_agg[df_agg['antall_kollisjoner']<10].copy()

In [518]:
df_agg["antall_kollisjoner"].max()

10

In [534]:
model_nb = smf.glm(
    formula="antall_kollisjoner ~ C(årstid)",
    data=df_agg,
    family=sm.families.NegativeBinomial(),
    offset=df_agg["log_eksponering"]
).fit()

print(model_nb.summary())



                 Generalized Linear Model Regression Results                  
Dep. Variable:     antall_kollisjoner   No. Observations:                  475
Model:                            GLM   Df Residuals:                      471
Model Family:        NegativeBinomial   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -935.88
Date:                Tue, 03 Feb 2026   Deviance:                       379.87
Time:                        14:41:51   Pearson chi2:                 7.56e+03
No. Iterations:                     8   Pseudo R-squ. (CS):            0.01910
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -9.5640    

C:\Users\jens.nilsen\python\WPy64-31241\python-3.12.4.amd64\Lib\site-packages\statsmodels\genmod\families\family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [535]:
model_nb = smf.glm(
    formula="antall_kollisjoner ~ C(årstid)",
    data=df_agg,
    family=sm.families.Poisson(),
    offset=df_agg["log_eksponering"]
).fit()

print(model_nb.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:     antall_kollisjoner   No. Observations:                  475
Model:                            GLM   Df Residuals:                      471
Model Family:                 Poisson   Df Model:                            3
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -940.76
Date:                Tue, 03 Feb 2026   Deviance:                       830.97
Time:                        14:42:07   Pearson chi2:                 1.34e+04
No. Iterations:                     6   Pseudo R-squ. (CS):            0.05406
Covariance Type:            nonrobust                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept             -10.1739    

In [530]:
import numpy as np

def lag_arstidsjustering(model):
    """
    Lag justeringsfaktorar for årstid frå ein statsmodels GLM (NB / Poisson).

    Referansekategori får faktor 1.0.
    """
    params = model.params

    # Finn alle årstids-koeffisientar
    arstid_params = {
        k: v for k, v in params.items()
        if k.startswith("C(årstid)")
    }

    # Referanse (den som ikkje er i params)
    arstid_justering = {"Haust": 1.0}

    # Legg til dei estimerte årstidene
    for k, beta in arstid_params.items():
        # Trekk ut årstidsnamn, t.d. C(årstid)[T.Sommar] -> Sommar
        arstid = k.split("[T.")[1].rstrip("]")
        arstid_justering[arstid] = float(np.exp(beta))

    return arstid_justering


In [531]:
ARSTID_JUSTERING = lag_arstidsjustering(model_nb)


In [532]:
ARSTID_JUSTERING

{'Haust': 1.0,
 'Sommar': 0.6516269675329989,
 'Vinter': 1.0553880422670336,
 'Vår': 0.7021711160218487}

In [533]:
(
    df_agg
    .groupby("årstid", observed=True)["antall_kollisjoner"]
    .sum()
    .sort_values(ascending=False)
)


årstid
Vinter    274
Haust     227
Vår       101
Sommar     91
Name: antall_kollisjoner, dtype: int64

In [525]:
(
    df
    .groupby("årstid", observed=True)
    .size()
    .sort_values(ascending=False)
)


årstid
Vinter    274
Haust     227
Vår       101
Sommar     91
dtype: int64

In [526]:
(
    df_agg
    .groupby("årstid", observed=True)["antall_kollisjoner"]
    .sum()
    .pipe(lambda s: s / s.sum())
)


årstid
Haust     0.327561
Sommar    0.131313
Vinter    0.395382
Vår       0.145743
Name: antall_kollisjoner, dtype: float64

In [527]:
(
    df
    .groupby("årstid", observed=True)
    .apply(
        lambda g: g.shape[0] / g["eksponering"].sum()
    )
    .sort_values(ascending=False)
)


C:\Users\jens.nilsen\AppData\Local\Temp\ipykernel_28456\484447156.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


årstid
Haust     0.000021
Sommar    0.000020
Vinter    0.000019
Vår       0.000017
dtype: float64